In [1]:
# попытка автоматизации программы с вводом исключительно периода за который берутся измерения

import numpy as np
import copy
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from ftplib import FTP
import os
from datetime import datetime
from pathlib import Path
import subprocess
import shutil
import pandas as pd
from datetime import timedelta
from scipy.signal import butter, filtfilt, decimate, lfilter, welch, firwin, cheb1ord, cheby1, remez, sosfilt
import piecewise_regression
import pywt
import scaleogram as scg
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import colors
from scipy.stats import bootstrap
import aacgmv2

# mask_cut_off = 30.0
wn = 1 / 30
# time_window = 120 # делим время на ячейки
# lat_window = 20 # делим широту на ячейки
# time_floor = 0
# time_ceil = 24
# satellite_num = 0

# задание координат для отрисовки карты
extent = [10, 50, 40, 72]
central_lon, central_lat = np.mean(extent[:2]), np.mean(extent[2:])
land = cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='black', linewidth = .3, facecolor=cfeature.COLORS['land'])
ocean = cfeature.NaturalEarthFeature('physical', 'ocean', '10m', edgecolor='face', linewidth = .3, facecolor=cfeature.COLORS['water'])
saintp = dict(lon = 59.938670, lat = 30.314350)
svtl = dict(lon = 60.533, lat = 29.781)
mdvj = dict(lon = 56.021, lat = 37.215)
zeck = dict(lon = 43.788, lat = 41.565)
polv = dict(lon = 49.603, lat = 34.543)
kiru = dict(lon = 67.857, lat = 20.968)
riga = dict(lon = 56.949, lat = 24.059)
lama = dict(lon = 53.892, lat = 20.670)
uzhl = dict(lon = 48.632, lat = 22.298)
bucu = dict(lon = 44.464, lat = 26.126)
time_per_degree = 360/(24*3600)

directory = f'c:/Users/Даниил/Desktop/datalist/data/'
stations = ['svtl', 'zeck', 'krs1', 'glsv', 'mdvj', 'auto', 'SEPT', 'POLV', 'KIRU', 'LAMA', 'RIGA', 'UZHL','BUCU', 'kiru', 'bucu', 'polv', 'riga', 'uzhl'] # список станций (лучше вынести в txt)
coikw = {'alpha': 0.5, 'hatch': '/'}
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

# ---------------------------------------------------------------
# Технические функции, фильтры, перевод координат
     
def discret_detect(t1,t2):
    return round(1 / (time_lag(t1,t2) * 60))

def rolling_mean(data, window=20):
    return np.mean(data[-window:])

def convert_time_to_local(time, longitude):
    
    utc_time = datetime.strptime(time, '%H:%M:%S')
    time = datetime2decimals(utc_time)
    
    if longitude <= 0:
        time -= (abs(longitude) / time_per_degree) * (1/3600)
        if time < 0:
            time += 24.0
    else:
        time += (longitude / time_per_degree) * (1/3600)

    return time 

def time_lag(t_i1, t_i2):
    time_diff = 0
    if abs(t_i1) > abs(t_i2):
        time_diff = abs(t_i2) + 24.0 - abs(t_i1)
    else:
        time_diff = t_i2 - t_i1
    return np.round((np.floor(time_diff) + (time_diff % 1)), 5)

def datetime2decimals(time):
    hours = time.hour
    minutes = time.minute
    seconds = time.second
    decimal_time = hours + (minutes / 60) + (seconds / 3600)
    
    return decimal_time

def butter_lowpass(sampling_freq, wn, order = 5):
    nyq = 0.5 * wn
    # normal_cutoff = cutoff / nyq
    b, a = butter(order, nyq, btype='low', analog=False, fs = sampling_freq)
    return b, a

def lowpass_filter(data, sampling_freq, wn, order = 5):
    b, a = butter_lowpass(sampling_freq, wn, order = order)
    y = filtfilt(b, a, data)
    return y

def butterfilt(dict_tec):
    dict_tec = pd.DataFrame(dict_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    sampling_freq = 1 / (60 / discret_detect(dict_tec['time'][2], dict_tec['time'][3]))
    if sampling_freq > wn / 2:
        vtec= lowpass_filter(dict_tec['vtec'].values, sampling_freq, wn)
        dict_tec['vtec'] = vtec
    else:
        pass
    return dict_tec.values.tolist()

def cheb(dict_tec):
    dict_tec = pd.DataFrame(dict_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    sampling_freq = 1 / (60 / discret_detect(dict_tec['time'][2], dict_tec['time'][3]))
    if sampling_freq > wn / 2:
        N, W_n = cheb1ord(wn, (wn**-1 - 5)**-1, 0.5, 40, fs = sampling_freq)
        sos = cheby1(N, 5, W_n, fs = sampling_freq, output = 'sos')
        dict_tec['vtec'] = sosfilt(sos, dict_tec['vtec'])
        dict_tec = dict_tec.iloc[N+50:].reset_index(drop=True)
    else:
        pass
    return dict_tec.values.tolist()

def remezfilt(dict_tec):
    dict_tec = pd.DataFrame(dict_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    tapz = 128
    sampling_freq = 1 / (60 / discret_detect(dict_tec['time'][2], dict_tec['time'][3]))
    if sampling_freq > wn / 2:
        fir_coeffs = remez(numtaps = tapz, bands = [0, wn, (wn**-1 - 3)**(-1), 0.5 * sampling_freq], desired = [1, 0], fs = sampling_freq) 
        dict_tec['vtec'] = lfilter(fir_coeffs, [1], dict_tec['vtec'])
        # dict_tec['vtec'] = sosfilt(taps, dict_tec['vtec'])
        dict_tec = dict_tec.iloc[tapz:].reset_index(drop=True)
    else:
        pass
    return dict_tec.values.tolist()

def firwinfilt(dict_tec):
    dict_tec = pd.DataFrame(dict_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    sampling_freq = 1 / (60 / discret_detect(dict_tec['time'][2], dict_tec['time'][3]))
    if sampling_freq > wn / 2:
        taps = 128
        h = firwin(taps, wn, window=('kaiser', 9), pass_zero = True)
        vtec= lfilter(h, 1, dict_tec['vtec'].values)
        dict_tec['vtec'] = vtec
        dict_tec = dict_tec.iloc[taps:].reset_index(drop=True)
    else:
        pass
    return dict_tec.values.tolist()

def decimation(dict_tec):
    dict_tec = pd.DataFrame(dict_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    sampling_freq = 1 / (60 / discret_detect(dict_tec['time'][2], dict_tec['time'][3]))
    if sampling_freq > wn / 2:
        decimation_factor = int(np.ceil(sampling_freq * 30))
        while decimation_factor > 1:
            if decimation_factor > 13:
                vtec = decimate(dict_tec['vtec'].values, q = 10, ftype = 'fir', n = 4)
                dict_tec = dict_tec.iloc[::10]
                dict_tec['vtec'] = vtec        
                decimation_factor = decimation_factor // 10 
            else:
                vtec = decimate(dict_tec['vtec'].values, q = decimation_factor, ftype = 'iir', n = 8)
                dict_tec = dict_tec.iloc[::decimation_factor]
                dict_tec['vtec'] = vtec
                decimation_factor -= decimation_factor
            dict_tec = dict_tec.iloc[10:].reset_index(drop=True)    
    else:
        pass
    return dict_tec.values.tolist()

def decimation_2(data):
    sampling_freq = 60 / discret_detect(data['time'].tolist()[0], data['time'].tolist()[1])
    data = data.iloc[::int(30/sampling_freq)].reset_index(drop=True)  
    return data 

def phi_t(time):
    return (3.16 - 5.6 * np.cos(np.radians(15.0 *(time - 2.4))) + 1.4 * np.cos(np.radians(15.0 * (2 * time - 0.8))))
def phi_N(lamb):
    return (0.85 * np.cos(np.radians(lamb + 63.0)) - 0.52 * np.cos(np.radians(2.0 * lamb + 5.0)))
def phi_min_boundary(kp_m, time, lamb):
    return (65.5 - 2.4 * kp_m + phi_t(time) + phi_N(lamb) * np.exp(-0.3 * kp_m) )
def phi_min_polar_boundary(time, k):
    return (76.0 + 0.3 * k - (1 - 0.1 * k) * np.cos(np.radians(15.0 * time)))
def phi_avr(time, k):
    return (74.0) - 5.0 * np.cos(np.radians(15 * time)) - 1.4 * k

def geographic_to_magnetic_latitude(geo_latitude, geo_longitude, altitude, dtime):
    [magnetic_latitude, magnetic_longitude, _] = aacgmv2.get_aacgm_coord_arr(geo_latitude, geo_longitude, altitude, dtime, method = 'G2A')
    return magnetic_latitude, magnetic_longitude

filter_functions = {
    'butter': butterfilt,
    'cheby': cheb,
    'remez': remezfilt,
    'firwin': firwinfilt,
    'decimation': decimation
}  
    
# ---------------------------------------------------------
# Загрузка данных (если путь к отдельному файлу или в папку)

def load_data(file_path, mask_cut_off, time_floor, time_ceil, satellite_num):
    with open(file_path, 'r') as tec:
        tec = tec.readlines()
    tec = tec[5:]
    for i in range(len(tec)):
        tec[i] = list(map(float, tec[i].split()))[1:-1]    
    tec = pd.DataFrame(tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    tec = tec[(tec['el'] > mask_cut_off) & (tec['time'] > time_floor) & (tec['time'] < time_ceil) & (tec['vtec'] > 0)]
    if satellite_num in tec['satellite'].tolist():
        tec = tec[tec['satellite'] == satellite_num]
    tec = tec.values.tolist()
    return tec      

def load_data_path(data_path):
    dict_tec = {}
    p = Path(data_path)

    if not p.is_dir():
        raise ValueError(f"The path {data_path} is not a valid directory.")

    for station in stations:
        period_tec = []
        time_bias = timedelta(hours=0)
        
        found_data = False
        for CMN in sorted(os.listdir(data_path)):
            if CMN.startswith(station):
                found_data = True
                CMN_path = p / CMN
                
                try:
                    daily_data = load_data(CMN_path)
                except Exception as e:
                    print(f"Error loading data from {CMN_path}: {e}")
                    continue
                
                for entry in daily_data:
                    time_entry = entry[0] + time_bias.total_seconds() / 3600
                    new_entry = [time_entry] + entry[1:]
                    period_tec.append(new_entry)
                
                time_bias += timedelta(hours=24)

        if found_data:
            dict_tec[station] = pd.DataFrame(period_tec, columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])

    return dict_tec

def load_bz_data(bz_path):
    with open(bz_path, 'r') as bz:
        bz = bz.readlines()
    for i in range(len(bz)):
        bz[i] = list(map(float, bz[i].split()))  
    bz = pd.DataFrame(bz)
    bz = bz.filter(bz.columns[[0,1,2,3,13,16,18]])
    bz.columns = ['year', 'day', 'hour', 'minute', 'INF', 'bz(GSE)', 'bz(GSM)']
    bz = bz[(bz['bz(GSE)'] < 9999) & (bz['bz(GSM)'] < 9999)]
    bz['time'] = pd.to_timedelta(bz['hour'], unit='h') + pd.to_timedelta(bz['minute'], unit='m')
    bz['time'] = bz['time'].dt.components.apply(lambda x: f"{int(x.hours):02}:{int(x.minutes):02}", axis=1)
    bz['time_decimal'] = bz['hour'] + bz['minute']/60
    return bz

def load_iaga_data(ae_path, DoYs, DoYe):
    ae = pd.read_csv(ae_path,  sep='\s+', skiprows = 14)
    ae.drop('|', axis = 1, inplace = True)
    ae['TIME'] = pd.to_datetime(ae['TIME'], format='%H:%M:%S.%f')
    ae['TIME_UTC'] = pd.to_datetime(ae['TIME'], format='%H:%M:%S.%f').dt.strftime('%H:%M')
    ae['TIME_DECIMAL'] = ae['TIME'].dt.hour + ae['TIME'].dt.minute / 60
    ae.drop('TIME', axis = 1, inplace = True)
    ae = ae[(ae['DOY'] <= DoYe) & (ae['DOY'] >= DoYs)]
    return ae 

# Обработка данных (перевод в таблицу pandas, выделение отдельных проходов каждого спутника, применение фильтра к VTEC)
 
def process_data(tec, filter_name):
    dict_tec = {}
    filtered_dict_tec = {}
    tec = pd.DataFrame(tec, columns = ['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    
    for satellite, group in tec.groupby('satellite'):
        dict_tec[satellite] = group.values.tolist()
    
    for key in list(dict_tec.keys()):
        surrogate = []
        flag = 0
        data = dict_tec[key]
        
        for j in range(1, len(data)):
            if time_lag(data[j-1][0], data[j][0]) > wn * 1.7:
                surrogate.append(data[flag:j])
                flag = j
        
        if flag != 0:
            dict_tec[key] = surrogate + [data[flag:]]
        else:
            dict_tec[key] = [dict_tec[key]]
            
        filtered_sessions = []
        for session in dict_tec[key]: 
            if len(session) >= 100:
                if filter_name in filter_functions:
                    session = filter_functions[filter_name](session)
                filtered_sessions.append(session)
                
        dict_tec[key] = filtered_sessions
                
    return dict_tec
    
def process_data_path(dict_tec, filter_name):
    filtered_dict_tec = {}
    
    for station, df in dict_tec.items():
        filtered_dict_tec[station] = {}
    
        for satellite_id, group in df.groupby('satellite'):
            data_list = group.values.tolist()
            
            if satellite_id not in filtered_dict_tec[station]:
                filtered_dict_tec[station][satellite_id] = []
            filtered_dict_tec[station][satellite_id].extend(data_list)

    for key1 in filtered_dict_tec.keys():
        for key2 in filtered_dict_tec[key1].keys():
            surrogate = []
            flag = 0
            data = filtered_dict_tec[key1][key2][:-10]
            
            for j in range(1, len(data)):
                if time_lag(data[j-1][0], data[j][0]) > wn * 1.7:
                    surrogate.append(data[flag:j])
                    flag = j
            
            if flag != 0:
                filtered_dict_tec[key1][key2] = surrogate + [data[flag:]]
            else:
                filtered_dict_tec[key1][key2] = [filtered_dict_tec[key1][key2]]
            
    for key1 in filtered_dict_tec.keys():
        for key2 in filtered_dict_tec[key1].keys():
            
            filtered_sessions = []
            for session in filtered_dict_tec[key1][key2]: 
                if len(session) >= 100:
                    # print(session)
                    if filter_name in filter_functions:
                        session = filter_functions[filter_name](session)
                    filtered_sessions.append(session)
                    
            filtered_dict_tec[key1][key2] = filtered_sessions
                
    return filtered_dict_tec

# Сохранение данных (.csv файл для отдельного прохода где только время и ТЕС)

def save_data(dict_tec, output_directory, file_path,ds, satellite_num, filter_name):
    time_surr, tec_surr = [],[]
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            # if filter_name in filter_functions:
            #     dict_tec[t][count] = filter_functions[filter_name](dict_tec[t][count])    
            time, tec = graph_tec(dict_tec, t, count)
            time_surr.extend(time)
            tec_surr.extend(tec)
    day_download = ds.strftime('%d-%B-%Y')              
    data = pd.DataFrame({'time': time_surr, 'tec': tec_surr})
    mean_value = data.groupby('time')['tec'].mean().reset_index()
    data = data.drop(columns = ['tec']).drop_duplicates(subset=['time']).merge(mean_value, on ='time')  
    data = data.sort_values(by='time').reset_index(drop=True)
    file_name = os.path.basename(file_path)
    filepath = os.path.dirname(output_directory + '/')
    csv_path = os.path.join(filepath, f'processed_tec{file_name[:4]}{satellite_num}-{day_download}-{filter_name}.csv')
    return data.to_csv(csv_path, index = False, sep =' ', na_rep = '')

# Подсчет ROTI (а также усредненное за 5 минут время и координаты подыоносферной точки)
 
def graph_roti(dict_tec, t, count):
    rot, time, latitude = [], [], []
    dict_tec1 = pd.DataFrame(copy.copy(dict_tec[t][count]), columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    dict_tec1 = dict_tec1.drop_duplicates(subset=['time']).sort_values(by='time').reset_index(drop=True)
    # fs = 1 / (np.mean(np.diff(dict_tec1['time'])) * 3600)
    fs = 1 / (60 / discret_detect(dict_tec1['time'].tolist()[0], dict_tec1['time'].tolist()[1]))
    # wn_roti = 1 / 61
    # dict_tec1['vtec'] = lowpass_filter(dict_tec1['vtec'], fs, wn_roti, order = 5)
    # dict_tec1[t][count] = dict_tec1[t][count][::discret_detect(dict_tec1[t][count][0][0], dict_tec1[t][count][1][0])]
    dict_tec1 = dict_tec1.iloc[::int(round(60 * fs))].reset_index(drop=True) 
    rot = np.abs(dict_tec1['vtec'].diff().iloc[1:])  
    time = dict_tec1['time'].iloc[1:] 
    latitude = dict_tec1['lat'].iloc[1:]
    longitude = dict_tec1['lon'].iloc[1:] 
    
    roti, time_mean, latitude_mean, longitude_mean = [], [], [], []

    for i in range(0, len(rot) - 4):
        roti.append(np.std(rot[i:i+5]))
        time_mean.append(np.median(time.iloc[i: i+5])) 
        latitude_mean.append(np.median(latitude.iloc[i:i+5]))
        longitude_mean.append(np.median(longitude.iloc[i:i+5]))
    if len(time_mean) != len(roti):
        raise ValueError("Length of time_mean and roti must be the same")
    return [time_mean, roti, latitude_mean, longitude_mean]

# Выделение ТЕС и времени и построение графиков ТЕС, ROTI

def graph_tec(dict_tec, t, count):
    df = pd.DataFrame(dict_tec[t][count], columns = ['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
    time = df['time'].tolist()
    vstec = df['vtec'].tolist()
           
    return [time, vstec]

def plot_graph_tec_path(dict_tec, ds):
        for key in dict_tec.keys():
            tec = dict_tec[key]
            plot_graph_tec(tec, ds)

def plot_graph_roti(dict_tec, ds, static_folder):
    graph_num = sum(len(value) for value in dict_tec.values())
    colors = [plt.get_cmap('gist_ncar')(float(i / graph_num)) for i in range(graph_num)]

    fig, ax = plt.subplots()
    count_index = 0
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time, roti, latitude, _ = graph_roti(dict_tec, t, count)
            ax.plot(time, roti, c = colors[count_index], linewidth = 0.63)
            count_index += 1
            
    ax.set_xlabel('Время, часы')
    ax.set_ylabel('ROTI, TECU/мин')
    # plt.title(f'ROTI {ds}')
    plt.tight_layout()
    plot_filename = f'roti_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename

def plot_graph_roti_path(dict_tec, ds):
    for key in dict_tec.keys():
        plot_graph_roti(dict_tec[key], ds)
    
def plot_graph_tec(dict_tec, ds, static_folder):
    graph_num = sum(len(value) for value in dict_tec.values())
    colors = [plt.get_cmap('gist_ncar')(float(i / graph_num)) for i in range(graph_num)]

    fig, ax = plt.subplots()
    count_index = 0
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time, tec = graph_tec(dict_tec, t, count)
            ax.plot(time, tec, c = colors[count_index], linewidth = 0.63, label = f'{t} satellite {count+1} session')
            count_index += 1
            # plt.text(time[0], tec[0], f'{t} satellite, {count+1} session', 
            # fontsize = 6)
            
    ax.set_xlabel('Время, часы')
    ax.set_ylabel('TEC, TECu')
    # plt.title(f'TEC {ds}')
    # plt.legend()
    if len(dict_tec) < 15:
        plt.legend(loc = 'upper center', fontsize = 'xx-small', bbox_to_anchor=(1, 1))
    # plt.tight_layout()
    plot_filename = f'tec_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename
    
# Спектры ТЕС через Уэлча (для каждого прохода отдельно)
    
def plot_tec_spectrogram(dict_tec, key, static_folder):
    graph_num = sum(len(value) for value in dict_tec.values())
    colors = [plt.get_cmap('gist_ncar')(float(i / graph_num)) for i in range(graph_num)]
    
    fig, ax = plt.subplots()
    count_index = 0
    Nsub = 10
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time, vstec = graph_tec(dict_tec, t, count)
            # Nfft = len(vstec) - 1
            nperseg = len(vstec) / Nsub
            if len(time) > 100:
                sampling_freq = 1 / (60 / discret_detect(time[1], time[2]))
                f, Pxx_den = welch(vstec, fs = sampling_freq, nperseg = len(vstec)/Nsub, noverlap=nperseg/2, scaling='spectrum', nfft=nperseg, window= ('kaiser', 14), detrend = 'linear')
                if len(dict_tec) <= 15:
                    try:
                        pw_fit = piecewise_regression.Fit(np.log10(f[1:]), np.log10(Pxx_den[1:]), n_breakpoints=1)
                        # pw_fit.plot()
                        print(f'{t} satellite {count+1} session')
                        pw_results = pw_fit.get_results()
                        print('alpha1' ,pw_results['estimates']['alpha1'])
                        print('breakpoint' ,pw_results['estimates']['breakpoint1'])
                    except TypeError:
                        print('Linear regression failed')
                # f, Pxx_den = welch(vstec, fs = sampling_freq, noverlap=0, nperseg = len(vstec)/Nsub, scaling='spectrum', window='hamming')
                plt.semilogy(f, Pxx_den, linewidth = 0.63, label = f'{t} satellite {count+1} session', c = colors[count_index])
                count_index += 1
    # plt.legend()
    plt.ylabel('Спектральная плотность мощности, дБ')
    plt.xlabel('Частота, Гц')
    plt.xscale('log')
    # plt.legend(loc='best')
    # plt.text(time[0], vstec[0], f'{t} satellite, {count+1} session', 
    # fontsize = 6)
    if len(dict_tec) < 15:
        plt.legend(loc='upper center', bbox_to_anchor=(1, 1), fontsize = 'xx-small')
    plt.title(key)
    plt.grid(True)
    # plt.tight_layout()
    plot_filename = f'spec_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename
    
def plot_tec_spectrogram_path(dict_tec):
    for key in dict_tec.keys():
        plot_tec_spectrogram(dict_tec[key], key)

# Общий спектр ТЕС (сделана синтетическая последовательность величин ТЕС через усреднение данных по всем спутникам для одного момента времени)

def plot_tec_spec_once(tec_data):
    fig, ax = plt.subplots()
    count_index = 0
    Nsub = 10
    for key in tec_data.keys():
        time = tec_data[key]['time'].tolist()
        vstec = tec_data[key]['vtec'].tolist()
        Nfft = len(vstec) - 1
        nperseg = len(vstec) / Nsub
        if len(time) > 100:
            sampling_freq = 1 / (60 / discret_detect(time[1], time[2]))
            f, Pxx_den = welch(vstec, fs = sampling_freq, nperseg = len(vstec)/Nsub, noverlap=nperseg/2, scaling='spectrum', nfft=nperseg, window='hamming')
            # f, Pxx_den = welch(vstec, fs = sampling_freq, noverlap=0, nperseg = len(vstec)/Nsub, scaling='spectrum', window='hamming')
            pw_fit = piecewise_regression.Fit(np.log10(f[1:]), np.log10(Pxx_den[1:]), n_breakpoints=1)
            pw_fit.plot()
            pw_fit.summary()
            # plt.show()
            # plt.semilogy(f, Pxx_den, linewidth = 0.63)
            count_index += 1
        # plt.legend()
        plt.ylabel('Спектральная плотность мощности, дБ')
        plt.xlabel('Частота, Гц')
        plt.xscale('log')
        # plt.legend(loc='best')
        plt.title(key)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

def plot_bz(bz, DoYs, DoYe, dict_tec):
    bz = bz[(bz['day'] >= DoYs) & (bz['day'] <= DoYe) & (bz['time_decimal'] >= dict_tec[0]) & (bz['time_decimal'] <= dict_tec[-1])]
    return bz
    
# Кеограмма
 
def plot_roti_heatmap(dict_tec, datez, window_size, lat_window, static_folder):
    time_mean, latitude_mean, roti, longitude_mean = [], [], [], [] 
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time_surr, roti_surr, latitude_surr, longitude_surr = graph_roti(dict_tec, t, count)
            time_mean.extend(time_surr)
            latitude_mean.extend(latitude_surr)
            longitude_mean.extend(longitude_surr)
            roti.extend(roti_surr)
    
    dtime = datetime.strptime(datez[0], "%d.%m.%Y")
    mlat, mlon = geographic_to_magnetic_latitude(latitude_mean, longitude_mean, 350, dtime)    
    time_bins = np.linspace(min(time_mean), max(time_mean), num = int(window_size))
    lat_bins = np.linspace(min(mlat), max(mlat), num = lat_window)
    # H, xedges, yedges = np.histogram2d(time_mean, latitude_mean, bins = [time_bins, lat_bins], weights = roti)
    # H_avg = H.T  
    H_counts, xedges, yedges = np.histogram2d(time_mean, mlat, bins=[time_bins, lat_bins])
    H_sums, _, _ = np.histogram2d(time_mean, mlat, bins=[time_bins, lat_bins], weights=roti)
    H_avg = np.divide(H_sums, H_counts, out=np.zeros_like(H_sums), where=H_counts != 0).T  # 
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    
    fig = plt.figure(figsize=(10, 6))
    imshow = plt.imshow(H_avg, extent = extent, origin = 'lower', cmap = 'jet', aspect = 'auto', interpolation = 'none')
    plt.xlabel('Время, часы')
    plt.ylabel('MLat, градусы')
    plt.title(f'ROTI Кеограмма с \n {((max(time_mean)-min(time_mean)) * 60) // window_size} мин в клетке {((max(mlat)-min(mlat))) // lat_window} градусов на клетку')
    plt.grid(True)
    plt.colorbar(label='Интенсивность ROTI')
    plot_filename = f'keo_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename

def plot_roti_heatmap_path(dict_tec):
    filt_tec = {}
    for station, data in dict_tec.items():
        filt_tec.update({str(satellite) + str(station): value for satellite, value in data.items()})
    plot_roti_heatmap(filt_tec)

# Построение вейвлет-спектрограммы
 
def plot_wavelet(dict_tec, filter_name, flaglist, static_folder):
    time_surr, tec_surr = [],[]
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time, tec = graph_tec(dict_tec, t, count)
            time_surr.extend(time)
            tec_surr.extend(tec)
                   
    data = pd.DataFrame({'time': time_surr, 'tec': tec_surr})
    mean_value = data.groupby('time')['tec'].mean().reset_index()
    data = data.drop(columns = ['tec']).drop_duplicates(subset=['time']).merge(mean_value, on ='time')  
    data = data.sort_values(by='time').reset_index(drop=True)
    if ((filter_name != 'decimation') and ('DEC' in flaglist)):   
        data = decimation_2(data).reset_index(drop=True) 
    
    # scales = np.geomspace(1, 1024, num=100)
    # sampling_freq = 1 / (60 / discret_detect(data['time'].tolist()[2], data['time'].tolist()[3]))
    # coeff, freq = pywt.cwt(data['tec'], scales, wavelet = 'cmor0.5-1.0', sampling_period=sampling_freq)
    # coeff = np.abs(coeff[:-1, :-1])
    # period = 1. / freq
    # plt.figure()
    
    scales = scg.periods2scales(np.arange(1/15,3,0.1))
    scg.cws(time = np.array(data['time'].tolist()),signal = data['tec'], scales=np.arange(50,256,0.1), coikw=coikw, cmap="jet", cbarlabel='PSD', ylabel='Период', xlabel="Время", yscale="log", title='CWT', wavelet = 'cmor3-1.5', spectrum = 'power') 
    
    # смотреть тут https://github.com/alsauve/scaleogram/blob/master/lib/scaleogram/cws.py
    # spectrum = 'power' for abs(CWT)**2
    # plt.imshow(coeff, extent=[data['time'].tolist()[0], data['time'].tolist()[-1], period[-1], period[0]], aspect='auto',
    #         interpolation='bicubic', cmap='viridis')
    # plt.colorbar(label='Абс. Величина')
    # plt.ylabel('Period')
    # plt.xlabel('Time (s)')
    # plt.yscale('log')
    # plt.ylim(0, max(freq))
    plt.title('Скалограмма')
    plot_filename = f'wt_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    # plt.close(fig)
    return plot_filename

def plot_wavelet_path(dict_tec):
    for key in dict_tec.keys():
        plot_wavelet(dict_tec[key])

# Спектрограмма ТЕС с графиками геомагнитных индексов
 
def plot_sfft_spec(dict_tec, bz, ae, sym, window_size, DoYs, DoYe, filter_name, flaglist, static_folder):
    n = 4
    l = [bz,ae,sym]
    size_plot = [5, 1, 1, 1]
    for ind in l:
        if len(ind) == 0:
            n -= 1
            size_plot.pop(-1)
    time_surr, tec_surr = [],[]
    for t in list(dict_tec.keys()):
        for count in range(len(dict_tec[t])):
            time, tec = graph_tec(dict_tec, t, count)
            time_surr.extend(time)
            tec_surr.extend(tec)
                   
    data = pd.DataFrame({'time': time_surr, 'tec': tec_surr})
    mean_value = data.groupby('time')['tec'].mean().reset_index()
    data = data.drop(columns = ['tec']).drop_duplicates(subset=['time']).merge(mean_value, on ='time')  
    data = data.sort_values(by='time').reset_index(drop=True)
    if ((filter_name != 'decimation') and ('DEC' in flaglist)):
        data = decimation_2(data) 
    window = time_lag(min(data['time']), max(data['time'])) 
    sampling_freq = 1 / (60 / discret_detect(data['time'][1], data['time'][2]))
    
    fig, axs = plt.subplots(n,1, layout='constrained', figsize=(16,9), gridspec_kw={'height_ratios': size_plot})
    NFFT = int(round(len(data['tec'].tolist()) // ((window * 60)) * window_size)) 
    Sxx, frequencies, bins, im = axs[0].specgram(data['tec'], Fs=sampling_freq, mode = 'psd', NFFT = NFFT, detrend = 'linear', noverlap = NFFT // 2, cmap = 'inferno')
    axs[0].set_title('SFFT Спектрограмма', fontsize=12)
    axs[0].set_ylabel('Частота, Гц', fontsize=8)
    # ax1.set_yscale('log')
    axs[0].set_ylim(0, max(frequencies))
    # num_ticks = 10 
    # tick_positions = bins[::len(bins)//num_ticks]  
    # tick_labels = [f"{int(np.floor(x))}:{str(round(x % 1 * 60)).zfill(2)}" for x in data['time'].tolist()[::len(data['time'].tolist()) // num_ticks]]
    # if len(tick_positions) != len(tick_labels):
    #     min_length = min(len(tick_positions), len(tick_labels))
    #     tick_positions = tick_positions[:min_length]
    #     tick_labels = tick_labels[:min_length]
    # axs[0].set_xticks(ticks=tick_positions, labels=tick_labels)
    axs[0].set_xticks([])
    
    if len(bz) > 0:
        bz = plot_bz(bz, DoYs, DoYe, data['time'].tolist())
        axs[1].plot(bz['time'], bz['bz(GSE)'])
        axs[1].set_ylabel('Bz, нТ',fontsize=8)
        # axs[1].set_xlabel('Время', fontsize=8)
        axs[1].set_xticks([])
        axs[1].set_xlim(min(bz['time']), max(bz['time']))
    
    if len(ae) > 0:    
        ae = ae[(ae['TIME_DECIMAL'] >= data['time'].tolist()[0]) & (ae['TIME_DECIMAL'] <= data['time'].tolist()[-1])]
        axs[2].plot(ae['TIME_UTC'], ae['AE'])
        axs[2].set_ylabel('AE, нT',fontsize=8)
        # axs[2].set_xlabel('Время', fontsize=8)
        axs[2].set_xticks([])
        axs[2].set_xlim(min(ae['TIME_UTC']), max(ae['TIME_UTC']))
    
    if len(sym) > 0:
        sym = sym[(sym['TIME_DECIMAL'] >= data['time'].tolist()[0]) & (sym['TIME_DECIMAL'] <= data['time'].tolist()[-1])]
        axs[3].plot(sym['TIME_UTC'], sym['SYM-H'])
        axs[3].set_ylabel('SYM-H, нT',fontsize=8)
        axs[3].set_xlabel('Время', fontsize=8)
        # axs[3].set_xticks([])
        axs[3].set_xlim(min(sym['TIME_UTC']), max(sym['TIME_UTC']))
    
    axs1 = plt.gca()
    axs1.xaxis.set_major_locator(MaxNLocator(nbins=10))
    cbar = fig.colorbar(axs[0].images[0], ax = axs,  orientation='vertical', fraction=0.1)
    # cbar = fig.colorbar(ax1.images[0], ax = (ax1, ax2),  location = 'bottom', fraction=0.07)
    cbar.set_label('Спектральная плотность мощности')
    # lmin = np.min(Sxx)
    # lmax = np.max(Sxx)
    # num_ticks = 5
    # ticks = np.linspace(lmin, lmax, num_ticks)
    # cbar.set_ticks(ticks)
    # cbar.minorticks_on()
    plot_filename = f'sfft_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename
    
def plot_sfft_spec_path(dict_tec):
    for key in dict_tec.keys():
        plot_sfft_spec(dict_tec[key])
        
# Построение карты с поточечным ROTI (где указывается координата подыоносферной точки)
 
def current_roti(narrow_dict, satellite, count, vremya):
    # utc = datetime.strptime(vremya, '%H:%M:%S')
    roti_time = datetime2decimals(vremya)
    rot, time, latitude = [], [], []
    rot = np.abs(narrow_dict['vtec'].diff().iloc[1:]) 
    time = np.mean(narrow_dict['time']) 
    latitude = np.mean(narrow_dict['lat'])
    longitude = np.mean(narrow_dict['lon'])
    roti = np.std(rot)
    return [roti, longitude, latitude, time]
    
def current_roti_scatter(vremya, dict_tec):
    roti_time = datetime2decimals(vremya)
    roti_surr, lon_surr, lat_surr, time_surr = [],[],[],[]
    for satellite in list(dict_tec.keys()):
        for count in range(len(dict_tec[satellite])):
            dict_tec1 = pd.DataFrame(copy.copy(dict_tec[satellite][count]), columns=['time', 'satellite', 'az', 'el', 'lat', 'lon', 'stec', 'vtec'])
            dict_tec1 = dict_tec1.drop_duplicates(subset=['time']).sort_values(by='time').reset_index(drop=True)
            # fs = 1 / (np.mean(np.diff(dict_tec1['time'])) * 3600)
            fs = 1 / (60 / discret_detect(dict_tec1['time'].tolist()[0], dict_tec1['time'].tolist()[1]))
            dict_tec1 = dict_tec1.iloc[::int(round(60 * fs))].reset_index(drop=True)
            narrow_dict = dict_tec1[(dict_tec1['time'] >= roti_time - (1/60*3)) & (dict_tec1['time'] <= roti_time + (1/60*3))]
            if narrow_dict[['vtec', 'time', 'lat', 'lon']].isnull().values.any():
                continue
            roti, longitude, latitude, time = current_roti(narrow_dict, satellite, count, vremya)
            if not np.isnan(roti):
                time_surr.append(time)
                lon_surr.append(longitude)
                lat_surr.append(latitude)
                roti_surr.append(roti)
    
    return [time_surr, lon_surr, lat_surr, roti_surr]

def plot_current_roti(utc, dict_tec, kp_m, utc_time, static_folder):
    utc_datetime = datetime.strptime(utc, '%H:%M:%S')
    time_surr, lon_surr, lat_surr, roti_surr = current_roti_scatter(utc_datetime, dict_tec)
    geo_dots = 361
    lambds = np.linspace(-180, 180, geo_dots)
    date = datetime(2024,5,10,utc_datetime.hour)
    times = [convert_time_to_local(utc, lamb) for lamb in lambds]
    tau = 0.6
    # kp_m = (1 - tau)*(9 + 8*tau + 8*tau**2 + 4*tau**3 + 3*tau**4)
    k = 1.2 * kp_m - 1.0
    if k > 8:
        k = 8
    boundary, peak_aurora = [], []
    boundary_polar = [phi_min_polar_boundary(timie, k) for timie in times]
    for i in range(len(lambds)):
        boundary.append(phi_min_boundary(kp_m, times[i], lambds[i]))
        peak_aurora.append(phi_avr(times[i], k))
    orthograph = ccrs.Orthographic(central_lon, central_lat)
    orthograph._threshold /= 10    
    fig = plt.figure()
    ax = plt.axes(projection=orthograph)
    # ax = plt.axes(projection=ccrs.Orthographic())
    # ax.set_global()
    ax.set_extent(extent)
    ax.gridlines(color ='black', linewidth = .1)
    ax.add_feature(ocean, zorder = 1)
    # ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(land)
    # ax.add_feature(cfeature.nightshade.Nightshade(time, alpha=0.2))
    ax.coastlines(resolution = '50m', color =  'black', linewidth = .01)
    for location in [saintp, svtl, mdvj, zeck, polv, kiru, riga, lama, uzhl, bucu]:
        ax.plot(location['lat'], location['lon'], 'rx', markersize=1.5, zorder = 3,transform=ccrs.Geodetic())
    roti_surr_array = np.array(roti_surr)
    alpha = np.where(roti_surr_array < 0.05, 0, 0.9)
    scatter = ax.scatter(lon_surr,lat_surr, s = roti_surr_array * 50, c = roti_surr_array, cmap = 'jet', marker = 'o',norm = colors.Normalize(vmin = 0, vmax = 0.2),alpha = alpha, transform = ccrs.PlateCarree(), zorder = 7)
    ax.plot(lambds, boundary_polar, 'g', linewidth=.5, zorder = 4, transform=ccrs.Geodetic())
    ax.plot(lambds, boundary, 'g', linewidth=.5, zorder = 5, transform=ccrs.Geodetic())
    ax.plot(lambds, peak_aurora, 'b', linewidth=.5, zorder = 6, transform=ccrs.Geodetic())
    plt.title(f'UTC {utc_time} \n ROTI')
    cbar = plt.colorbar(scatter, ax=ax,  orientation='vertical', fraction=0.1)
    cbar.set_label('ROTI, TECu/мин')
    plot_filename = f'proti_graph.png'
    plot_filepath = os.path.join(static_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)
    return plot_filename
    
def plot_current_roti_path(utc_time, dict_tec):
    filt_tec = {}
    for station, data in dict_tec.items():
        filt_tec.update({str(satellite) + str(station): value for satellite, value in data.items()})
    plot_current_roti(utc_time, filt_tec)

# Исполняющая функция

def main(file_path, bz_path, ae_path, sym_path, output_directory, datez, window_size, flaglist, satellite_num, wn, static_folder, time_ceil, time_floor, mask_cut_off, utc_time, lat_window, kp_m, filter_name):
    plot_filenames = []
    if len(datez) == 1:
            datez.append(datez[0])
    ds, de = datetime.strptime(datez[0], "%d.%m.%Y"), datetime.strptime(datez[1], "%d.%m.%Y")
    DoYs, DoYe = ds.timetuple().tm_yday, de.timetuple().tm_yday
    
    if file_path.endswith('.Cmn'):
        wn = wn
        tec_data = load_data(file_path, mask_cut_off, time_floor, time_ceil, satellite_num)
        processed_data = process_data(tec_data, filter_name)
        
        
        bz = load_bz_data(bz_path) if bz_path != None else []
        ae = load_iaga_data(ae_path) if ae_path != None else []
        sym = load_iaga_data(bz_path) if sym_path != None else []
        
        main_func = {
            "TEC": lambda: plot_graph_tec(processed_data, ds, static_folder),
            "SPEC": lambda: plot_tec_spectrogram(processed_data, file_path[file_path.rfind('\\')+1:-4], static_folder),
            "KEO": lambda: plot_roti_heatmap(processed_data, datez, window_size, lat_window, static_folder),
            "WT": lambda: plot_wavelet(processed_data, filter_name, flaglist, static_folder),
            "SFFT": lambda: plot_sfft_spec(processed_data, bz, ae, sym, window_size, DoYs, DoYe, filter_name, flaglist, static_folder),
            "SAVE": lambda: save_data(processed_data, output_directory, file_path, ds, satellite_num, filter_name),
            "PROTI": lambda: plot_current_roti(utc_time, processed_data, kp_m, utc_time, static_folder),
            "ROTI": lambda: plot_graph_roti(processed_data, ds, static_folder),
        }

        
        for flags in flaglist:
            if flags in main_func:
                plot_filenames.append(main_func[flags]()) 

    return plot_filenames

In [ ]:
import os
from flask import Flask, render_template, redirect, url_for, request
from flask_ngrok import run_with_ngrok
from flask_bootstrap import Bootstrap
import threading

app = Flask(__name__)
# bootstrap = Bootstrap(app)
# run_with_ngrok(app)

static_folder = os.path.join(app.root_path, 'static')

if not os.path.exists(static_folder):
    os.makedirs(static_folder)
    
UPLOAD_FOLDER = 'uploads'

if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def home():
    return render_template('input.html')

@app.route('/', methods = ['GET','POST']) 
def get_info():
    if request.method == 'POST':
        flaglist = request.form.getlist('options')
        rinex = request.files.get("rinex")
        bz = request.files.get("bz")
        ae = request.files.get("ae")
        sym = request.files.get("sym")

        rinex_path = os.path.join(app.config['UPLOAD_FOLDER'], rinex.filename)
        rinex.save(rinex_path)

        bz_path = os.path.join(app.config['UPLOAD_FOLDER'], bz.filename) if bz and bz.filename else None
        ae_path = os.path.join(app.config['UPLOAD_FOLDER'], ae.filename) if ae and ae.filename else None
        sym_path = os.path.join(app.config['UPLOAD_FOLDER'], sym.filename) if sym and sym.filename else None

        if bz_path:
            bz.save(bz_path)
        if ae_path:
            ae.save(ae_path)
        if sym_path:
            sym.save(sym_path)

        
        datez = str(request.form.get('doysdoye')).split()
        time_ceil = float(request.form.get('tend'))
        time_floor = float(request.form.get('tstart'))
        mask_cut_off = float(request.form.get('elang'))
        satellite_num = float(request.form.get('satnum'))
        window_size = int(request.form.get('winmin'))
        utc_time = str(request.form.get('timeproti'))
        lat_window = int(request.form.get('winlat'))
        kp_m = float(request.form.get('kp'))
        wn = float(request.form.get('wn'))
        filter_name = str(request.form.get('filt'))
        output_directory = ''
        
        plot_filenames = main(rinex_path, bz_path, ae_path, sym_path, output_directory, datez, window_size, flaglist, satellite_num, wn, static_folder, time_ceil, time_floor, mask_cut_off, utc_time, lat_window, kp_m, filter_name)
    
    return render_template('result.html', plot_filenames = plot_filenames)

def run():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.50.219:5000
Press CTRL+C to quit
192.168.50.219 - - [09/Feb/2025 16:29:43] "GET / HTTP/1.1" 200 -
192.168.50.219 - - [09/Feb/2025 16:29:52] "POST / HTTP/1.1" 200 -
192.168.50.219 - - [09/Feb/2025 16:29:52] "GET /static/tec_graph.png HTTP/1.1" 200 -
